In [1]:
read_from_gsheet = True

#only execute if we want to run the gsheet upload / download
if read_from_gsheet:
    !{sys.executable} -m pip install  pygsheets
    !{sys.executable} -m pip install --upgrade pyarrow
    !{sys.executable} -m pip install --upgrade google.cloud
    !{sys.executable} -m pip install --upgrade pandas_gbq

  Using cached pyarrow-6.0.1-cp38-cp38-macosx_10_13_x86_64.whl (19.1 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 8.0.0
    Uninstalling pyarrow-8.0.0:
      Successfully uninstalled pyarrow-8.0.0
  Attempting uninstall: pandas-gbq
    Found existing installation: pandas-gbq 0.17.5
    Uninstalling pandas-gbq-0.17.5:
      Successfully uninstalled pandas-gbq-0.17.5


In [2]:
#get data
import pygsheets
import configparser
import sys
import snowflake.connector
import pandas as pd
from datetime import datetime
import numpy as np                   # v 1.19.2
import matplotlib.pyplot as plt      # v 3.3.2
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
# calculate the net_arr bucket of open deals
import seaborn as sns
from math import floor 
from datetime import date

import numpy as np
import matplotlib.pyplot as plt

from gitlabdata.orchestration_utils import (
    data_science_engine_factory,
    query_dataframe,
)

engine = data_science_engine_factory()

service_file_path = 'nfiguera-c3fe9e64-a6543dd51e79.json'

#target gsheet for uploads and downloads, just allowing one to keep it more secure
spreadsheet_id = '19PPoHdc5nRZRX3dKGeGyYyTwqZ_x7GwD3EaDnwVm9Xs'

#how to add access to gsheet
#https://stackoverflow.com/questions/62917910/python-export-pandas-dataframe-to-google-sheets-solved

/Users/nfiguera/.local/share/virtualenvs/data-science-mn1T8veA/lib/python3.8/site-packages/snowflake/connector/options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.1), please install a version that adheres to: 'pyarrow<5.1.0,>=5.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

def read_from_gsheet(service_file_path, spreadsheet_id, sheet_name):
    """
    this function takes a sheet_name from a spreadsheet_id and returns a data frame 
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
   
    wks_read = sh.worksheet_by_title(sheet_name)
    read = wks_read.get_as_df()
    
    return read


def run_query_in_snowflake(conn, sql):
    cur = conn.cursor()
    cur.execute(sql)
    df = cur.fetch_pandas_all()
    return df

def executeScriptFromFile(filename, engine):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()
    
    results = -1
    
    try:
        results = query_dataframe(engine,sqlFile)
    except:
        print("Command did not run")
    
    return results

###########################################################

def days_between(d1, d2):
    #d1 = datetime.strptime(d1, "%Y-%m-%d")
    #d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days


def calculate_quarters_after_creation(x):
        
    age = 0
    
    if (x['IS_OPEN'] == 1):
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['SNAPSHOT_FISCAL_QUARTER_DATE'])
    elif (x['IS_OPEN']== 0 and x['SNAPSHOT_DATE'] <= x['CLOSE_DATE']):
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['SNAPSHOT_FISCAL_QUARTER_DATE'])
    else:
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['CLOSE_FISCAL_QUARTER_DATE'])
    
    quarter_delta = floor(age/90)
    
    return quarter_delta

def calculate_channel_track (x):
    
    channel_track = 'Direct'
    
    if (x['deal_path'] == 'Direct'):
        channel_track = 'Direct'
    elif (x['deal_path'] == 'Web Direct'):
        channel_track = 'Web Direct'
    elif (x['deal_path'] == 'Channel'
        and x['sales_qualified_source'] != 'Channel Generated'): 
        channel_track = 'Partner Co-Sell'
    elif (x['deal_path'] == 'Channel'):
        channel_track = 'Partner Sourced'
    
    return channel_track

def dt_inplace(df):
    """Automatically detect and convert (in place!) each
    dataframe column of datatype 'object' to a datetime just
    when ALL of its non-NaN values can be successfully parsed
    by pd.to_datetime().  Also returns a ref. to df for
    convenient use in an expression.
    """
    for c in df.columns[df.dtypes=='object']: #don't cnvt num
        try:
            df[c]=pd.to_datetime(df[c])
        except: #Can't cnvrt some
            try: 
                df[c]=df[c].astype(str)
            except:
                pass # ...so leave whole column as-is unconverted
            
    for c in df.columns[df.dtypes=='bool']: #don't cnvt num
        try: 
            df[c]=df[c].astype(int)
        except:
            pass # ...so leave whole column as-is unconverted
    return df

def join_column_to_df (df, source_df, key, key_source,target_column, rename_column):
    
    index = [key_source,target_column]
    df = df.merge(source_df[index],how='left',left_on=key,right_on=key_source)
    df.drop(key_source,axis=1,inplace=True)
    df.rename(columns={target_column:rename_column}, inplace=True)
    
    return df


In [4]:
#capture regions to country
#only execute if we want to run the gsheet upload / download
if read_from_gsheet:
    sheet_name = 'input_country_to_region'
    country_to_region = read_from_gsheet(service_file_path, spreadsheet_id, sheet_name)
 

In [5]:
#capture regions to country
#only execute if we want to run the gsheet upload / download
if read_from_gsheet:
    sheet_name = 'input_gitlab_industry_perc_it_spent'
    gitlab_industry_it_spent = read_from_gsheet(service_file_path, spreadsheet_id, sheet_name)

In [6]:
base_emea_accounts_summary = executeScriptFromFile('account_summary.sql', engine)
#base_emea_accounts_summary.to_csv('account_summary.csv')


Command did not run


In [7]:

emea_accounts_summary = base_emea_accounts_summary.merge(country_to_region[['sfdc_country','comm_region']],how='left',left_on='account_country',right_on='sfdc_country')
emea_accounts_summary.drop('sfdc_country',axis=1,inplace=True)
emea_accounts_summary.rename(columns={"comm_region":"account_region"}, inplace=True)

emea_accounts_summary = emea_accounts_summary.merge(country_to_region[['sfdc_country','comm_sub_region']],how='left',left_on='upa_country',right_on='sfdc_country')
emea_accounts_summary.drop('sfdc_country',axis=1,inplace=True)
emea_accounts_summary.rename(columns={"comm_sub_region":"upa_sub_region"}, inplace=True)

emea_accounts_summary = emea_accounts_summary.merge(country_to_region[['sfdc_country','comm_region']],how='left',left_on='upa_country',right_on='sfdc_country')
emea_accounts_summary.drop('sfdc_country',axis=1,inplace=True)
emea_accounts_summary.rename(columns={"comm_region":"upa_region"}, inplace=True)

emea_accounts_summary = emea_accounts_summary.merge(country_to_region[['sfdc_country','comm_region']],how='left',left_on='upa_country',right_on='sfdc_country')
emea_accounts_summary.drop('sfdc_country',axis=1,inplace=True)
emea_accounts_summary.rename(columns={"comm_region":"upa_fy23_region"}, inplace=True)

emea_accounts_summary['technical_account_manager_date'] = pd.to_datetime(emea_accounts_summary['technical_account_manager_date'])

#it industry spent
emea_accounts_summary = emea_accounts_summary.merge(gitlab_industry_it_spent[['gitlab_industry','it_spend_perc']],how='left',left_on='account_industry',right_on='gitlab_industry')
emea_accounts_summary.drop('gitlab_industry',axis=1,inplace=True)
emea_accounts_summary.rename(columns={"it_spend_perc":"account_it_spend_perc"}, inplace=True)
#replace nans with blanks
emea_accounts_summary[['upa_region','account_region','upa_fy23_region']] = emea_accounts_summary[['upa_region','account_region','upa_fy23_region']].replace(np.nan,'',regex=True)
emea_accounts_summary['account_potential_tam'] = emea_accounts_summary['zi_revenue'] * emea_accounts_summary['account_it_spend_perc'] * 0.1 * 0.2
emea_accounts_summary['technical_account_manager_date'] = pd.to_datetime(emea_accounts_summary['technical_account_manager_date'])



AttributeError: 'int' object has no attribute 'merge'

In [ ]:
#accounts_summaryDict = dict(emea_accounts_summary.dtypes)
#accounts_summaryDict
emea_accounts_summary.head(5)

In [ ]:
# import libraries
from google.oauth2 import service_account

# I like pandas gbq because it can infer schema
# https://pandas-gbq.readthedocs.io/en/latest/writing.html#inferring-the-table-schema
import pandas_gbq

# path to your json key file
KEY_PATH = "nfiguera_big_query.json"

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "nfiguera-c3fe9e64"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'nfiguera-c3fe9e64.proto.report_large_accounts_metrics'

# read the credentials from our file
# scopes are not necessary because we defined them in GCP already
CREDS = service_account.Credentials.from_service_account_file(KEY_PATH)

pandas_gbq.to_gbq(emea_accounts_summary, table_id, if_exists = 'replace', project_id=project_id, credentials = CREDS)

In [ ]:
len(emea_accounts_summary)

In [ ]:
sfdc_opportunity = executeScriptFromFile('sfdc_opportunity.sql', engine)
sfdc_opportunity.to_csv('sfdc_opportunity.csv')

dt_inplace(sfdc_opportunity)
#sfdc_opportunity_dict = dict(sfdc_opportunity.dtypes)
#sfdc_opportunity_dict

In [ ]:
# add start date per rep
sfdc_opportunity_new = join_column_to_df(sfdc_opportunity,country_to_region,'account_billing_country','sfdc_country','comm_region','account_region')
sfdc_opportunity_new = join_column_to_df(sfdc_opportunity_new,country_to_region,'upa_billing_country','sfdc_country','comm_region','upa_region')
sfdc_opportunity_new = join_column_to_df(sfdc_opportunity_new,country_to_region,'account_billing_country','sfdc_country','comm_sub_region','account_sub_region')
sfdc_opportunity_new = join_column_to_df(sfdc_opportunity_new,country_to_region,'upa_billing_country','sfdc_country','comm_sub_region','upa_sub_region')

In [ ]:
# import libraries
from google.oauth2 import service_account

# I like pandas gbq because it can infer schema
# https://pandas-gbq.readthedocs.io/en/latest/writing.html#inferring-the-table-schema
import pandas_gbq

# path to your json key file
KEY_PATH = "nfiguera_big_query.json"

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "nfiguera-c3fe9e64"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'nfiguera-c3fe9e64.proto.sfdc_opportunity_xf'

# read the credentials from our file
# scopes are not necessary because we defined them in GCP already
CREDS = service_account.Credentials.from_service_account_file(KEY_PATH)

pandas_gbq.to_gbq(sfdc_opportunity_new, table_id, if_exists = 'replace', project_id=project_id, credentials = CREDS)